In [1]:
import numpy as np
import onnxruntime as ort
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from io import BytesIO
from collections import namedtuple
from functools import partial
from hashlib import md5
from pathlib import Path
from typing import List, Union, Tuple, Dict
from tqdm.contrib.concurrent import process_map

from controllers import BaseController
from controllers import pid
from tinyphysics import TinyPhysicsModel, TinyPhysicsSimulator

import random

MODEL_PATH = 'models/tinyphysics.onnx'
DATA_PATH = 'data'


In [2]:
def run_rollout(data_path, controller, model_path, debug=False):
    tinyphysicsmodel = TinyPhysicsModel(model_path, debug=debug)
    sim = TinyPhysicsSimulator(tinyphysicsmodel, str(data_path), controller=controller, debug=debug)
    return sim.rollout(), sim.target_lataccel_history, sim.current_lataccel_history


In [19]:
genome = (0.12, 0.115, 0.005, 0.005)
def run_genome(genome, seg, model_path=MODEL_PATH, data_dir=DATA_PATH,):
    global controller
    data_dir = Path(data_dir)
    controller = pid.PID4Controller(*genome)
    file = data_dir / f"{seg:05d}.csv"
    print(file)
    return run_rollout(file, controller, model_path, debug=False)[0]


run_genome(genome, 0)

data\00000.csv


{'lataccel_cost': 1.503401287277064,
 'jerk_cost': 31.39390938185053,
 'total_cost': 106.56397374570373}

In [4]:
def runs_with_gains(genome, segs, model_path=MODEL_PATH, data_path=DATA_PATH):
    data_path = Path(data_path)
    controller = pid.PID4Controller(*genome)
    # run_rollout_partial = partial(run_rollout, controller_type=controller, model_path=model_path, debug=False)
    files = [data_path / f"{seg:05d}.csv" for seg in segs]
    # [print(str(file)) for file in files]
    # for f in files:
    #     print(f, f.is_file())
    # assert all([f.is_file() for f in files])
    # files = sorted(data_path.iterdir())[:args.num_segs]
    results = process_map(run_rollout_partial, files, max_workers=20)#, chunksize=10)
    costs = [result[0] for result in results]
    return costs

In [5]:
files_to_test = [random.randrange(5000, 20000 - 1) for _ in range(10)]
genome = (0.12, 0.115, 0.005, -0.0008)
run_with_gains(genome, files_to_test)

TypeError: unsupported format string passed to list.__format__